In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,8, 3, padding=1) 
        self.bnorm1 = nn.BatchNorm2d(8)

        self.conv2 = nn.Conv2d(8,8, 3, padding=1)
        self.bnorm2 = nn.BatchNorm2d(8)

        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(8, 16, 3, padding=1)
        self.bnorm3 = nn.BatchNorm2d(16)

        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
        self.bnorm4 = nn.BatchNorm2d(16)

        self.conv5 = nn.Conv2d(16, 16, 3, padding=1)
        self.bnorm5 = nn.BatchNorm2d(16)

        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv6 = nn.Conv2d(16, 32, 3)
        self.bnorm6 = nn.BatchNorm2d(32)
        
        self.conv7 = nn.Conv2d(32, 10, 3)
        self.dropout = nn.Dropout(0.2)
        self.aap = nn.AdaptiveAvgPool2d((1,1))

    def forward(self, x):
        x = self.pool1(self.bnorm2(F.relu(self.conv2(self.bnorm1(F.relu(self.conv1(x)))))))
        x = self.dropout(x)
        x = self.bnorm4(F.relu(self.conv4(self.bnorm3(F.relu(self.conv3(x))))))
        x = self.pool2((self.bnorm5(F.relu(self.conv5(x)))))
        x = self.dropout(x)
        x = self.bnorm6(F.relu(self.conv6(x)))
        x = self.conv7(x)
        x = self.dropout(x)
        x = self.aap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3            [-1, 8, 28, 28]             584
       BatchNorm2d-4            [-1, 8, 28, 28]              16
         MaxPool2d-5            [-1, 8, 14, 14]               0
           Dropout-6            [-1, 8, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           1,168
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 16, 14, 14]           2,320
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,320
      BatchNorm2d-12           [-1, 16, 14, 14]              32
        MaxPool2d-13             [-1, 16, 7, 7]               0
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0425, Accuracy: 9863/10000 (99%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0343, Accuracy: 9897/10000 (99%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0154, Accuracy: 9949/10000 (99%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0168, Accuracy: 9941/10000 (99%)



loss=0.004793298430740833 batch_id=937: 100%|██████████| 938/938 [00:30<00:00, 30.34it/s]



Test set: Average loss: 0.0178, Accuracy: 9944/10000 (99%)

